# <font color=#f4665a>  ICA analysis </font>

### Project: BMPD HC
____________________________________________________

**Description:** This notebook provides code for BOLD signal fMRI resting-state processing for the Biomarker for Parkinson's Disease (BMPD)data. 
We will used ICA analysis:
CanICA is an ICA package for group-level analysis of fMRI data.  
It brings a well-controlled group model, as well as a thresholding algorithm controlling for specificity and sensitivity with an explicit model of the signal.  
The reference papers are: G. Varoquaux et al. "A group model for stable multi-subject ICA on fMRI datasets", NeuroImage Vol 51 (2010), p. 288-299
G. Varoquaux et al. "ICA-based sparse features recovery from fMRI datasets", IEEE ISBI 2010, p. 1177


**Toolbox required:** SpinalCordToolbox, FSL, nilearn toolbox, nipype, matlab

**Inputs**:  
This notebook required this the following prepross anatomical,fmri images 

**Ouputs**:
See the output description at each step of the Notebook.

____________________________________________________


## <font color=#00988c> Imports </font>

In [2]:
import sys
import json
# Spinal cord Toolbox_________________________________________
### Cerebro:
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0/scripts") #sys.path.insert(0, "/cerebro/cerebro1/dataset/bmpd/derivatives/sc_preproc/code/sct/spinalcordtoolbox")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/") #sys.path.insert(0, "/cerebro/cerebro1/dataset/bmpd/derivatives/sc_preproc/code/sct/spinalcordtoolbox")

from spinalcordtoolbox.utils.sys import run_proc

from nilearn.maskers import NiftiMasker


from canICA_analyses import ICA
%matplotlib inline
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color=#00988c>  Run the ICA analysis </font>

In [3]:
# Load the dataset config
#config_spine_only_CL.json #../config/config_brsc_CL.json
with open('../config/config_spine_only_CL.json') as config_file:
    config = json.load(config_file)
dataset="gva" 
structures=["spinalcord"] # ["spinalcord"] or ["brain","spinalcord"] . double check the script for brainsc

In [ ]:
redo=True
config["data"][dataset]["inputs_ica"]["spinalcord"]['tag_filename_spinalcord']='moco_HP_sc_inTemplate_s.nii.gz'
for time in [1,2,3,4,5,6]:
    config["data"][dataset]['ica']["spinalcord"]['dir']='/ICA/results_spine_only/gva/spinalcord/temporal_cropping/' + str(time) + "min/"
    config['ica_ana']['iter']=1000
    config["ica_ana"]["k_range"]["spinalcord"]=[5]
    for k in config["ica_ana"]["k_range"]["spinalcord"]:
        config["ica_ana"]["n_comp"]=k # usefull if you want to test only on k
        print(config["ica_ana"]["n_comp"])
        icas = ICA(files_func[structure],[''],structures,dataset,config) # "brain_spinalcord" or "brain" or "spinalcord"
        #icas = ICA(files_func[structures[0]],files_func[structures[1]],structures,dataset,config) # "brain_spinalcord" or "brain" or "spinalcord"

        if k==4:
            all_data=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=1) # load or extract
        if redo==True:
            all_data=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
            all_data=icas.get_data(run='load',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
            reducedata_all=icas.indiv_PCA(all_data,save_indiv_img=True) # that step is not implanted to save individual maps for brain + sc yet
            components=icas.get_CCA(reducedata_all)
            components_final,components_final_z=icas.get_ICA(components)
            zcomponents4D_filename=icas.save_components(components_final,components_final_z)



In [39]:
redo=True
config["data"][dataset]["inputs_ica"]["spinalcord"]['tag_filename_spinalcord']='moco_HP_sc_inTemplate_s.nii.gz'
for iter in [100,200,500,1000,2000,5000]:
    config["data"][dataset]['ica']["spinalcord"]['dir']='/ICA/results_spine_only/gva/spinalcord/iterations_tests/' + str(iter) + "/"
    config['ica_ana']['iter']=iter
    config["ica_ana"]["k_range"]["spinalcord"]=[5]
    for k in config["ica_ana"]["k_range"]["spinalcord"]:
        config["ica_ana"]["n_comp"]=k # usefull if you want to test only on k
        print(config["ica_ana"]["n_comp"])
        icas = ICA(files_func[structure],[''],structures,dataset,config) # "brain_spinalcord" or "brain" or "spinalcord"
        #icas = ICA(files_func[structures[0]],files_func[structures[1]],structures,dataset,config) # "brain_spinalcord" or "brain" or "spinalcord"

        if k==4:
            all_data=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=1) # load or extract
        if redo==True:
            all_data=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
            all_data=icas.get_data(run='load',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
            reducedata_all=icas.indiv_PCA(all_data,save_indiv_img=True) # that step is not implanted to save individual maps for brain + sc yet
            components=icas.get_CCA(reducedata_all)
            components_final,components_final_z=icas.get_ICA(components)
            zcomponents4D_filename=icas.save_components(components_final,components_final_z)



['/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/Denoised_CL_DCT_NK//sub-01/spinalcord/sub-01_allvol.nii.gz',
 '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/Denoised_CL_DCT_NK//sub-02/spinalcord/sub-02_allvol.nii.gz',
 '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/Denoised_CL_DCT_NK//sub-03/spinalcord/sub-03_allvol.nii.gz',
 '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/Denoised_CL_DCT_NK//sub-04/spinalcord/sub-04_allvol.nii.gz',
 '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/Denoised_CL_DCT_NK//sub-05/spinalcord/sub-05_allvol.nii.gz',
 '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/Denoised_CL_DCT_NK//sub-06/spinalcord/sub-06_allvol.nii.gz',
 '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/Denoised_CL_DCT_NK//sub-07/spinalcord/sub-07_allvol.nii.gz',
 '/cerebro/cerebro1/